In [1]:
import os
import glob
import json
import time

from tqdm.notebook import tqdm

In [2]:
data_dir = 'data/'
train_file = 'data/train.jsonl'
dev_file = 'data/dev.jsonl'
wiki_pages_dir = 'data/wiki-pages/'

In [3]:
create_wiki_entity_map = True

## Creating entity map for entire wiki-database

In [4]:
wiki_pages = glob.glob(os.path.join(wiki_pages_dir, '*.jsonl'))

num_wiki_pages = len(wiki_pages)

In [6]:
for i in tqdm(range(num_wiki_pages)) : 
    
    wiki_file = open(wiki_pages[i])
    for line in wiki_file.readlines() : 
        data = json.loads(line)
        wiki_dict[data['id']] = wiki_pages[i]
    wiki_file.close()

  0%|          | 0/109 [00:00<?, ?it/s]

In [7]:
with open('data/entity-map.json', 'w') as f :
    json.dump(wiki_dict, f)

## Reading train.jsonl

In [8]:
with open(train_file) as f : 
    train_data = f.readlines()
    
num_train_data = len(train_data)

In [9]:
num_train_data

145449

In [10]:
def process_sample(sample) :
    
    processed = {}
    if sample['verifiable'] == 'VERIFIABLE' : 
        processed['claim'] = sample['claim']
        processed['label'] = sample['label']
        processed['evidence'] = {} 




        for evidence in sample['evidence'][0] : 

            wiki_file = open(wiki_dict[evidence[2]]) 

            for line in wiki_file.readlines() : 

                line = json.loads(line)

                if line['id'] == evidence[-2] : 

                    processed['evidence'][evidence[-2] + '_' + str(evidence[-1])] = {int(l.split('\t')[0]):l.split('\t')[1] for l in line['lines'].split('\n')}
                    
    return processed

In [11]:
processed_samples = []
num_supported = 0
num_refuted = 0 

max_num_supported = 25000
max_num_refuted = 25000

i = 0
start_time = time.time()


while not (num_supported==max_num_supported) and not (num_refuted==max_num_refuted) : 
    
    sample = json.loads(train_data[i])
    
    try : 
    
        if sample['verifiable'] == 'VERIFIABLE' : 
            if (sample['label'] == 'SUPPORTS') and not (num_supported==max_num_supported) : 
                
                processed = process_sample(sample)
                num_supported += 1
                processed_samples.append(processed)
            
            elif (sample['label'] == 'REFUTES') and not (num_refuted==max_num_refuted) :
                
                processed = process_sample(sample)
                num_refuted += 1
                processed_samples.append(processed)
            
        
    except Exception as e: 

        print(e)
        
    i += 1
    
    if i%500 == 0 : 
        
        print("Finished processing : {} samples.".format(i))
        print("There are {} samples that SUPPORTS.".format(num_supported))
        print("There are {} samples that REFUTES.".format(num_refuted))
        print("Time taken for last 500 iterations : " , time.time() - start_time)
        start_time = time.time()
    
    
with open('data/processed_samples.json', 'w') as f : 
    json.dump(processed_samples, f)

'José_María_Chacón'
'Régine_Chassagne'
'Chris_Pérez'
Finished processing : 500 samples.
There are 270 samples that SUPPORTS.
There are 97 samples that REFUTES.
Time taken for last 500 iterations :  119.0964138507843
'Beyoncé'
'Frédéric_Auguste_Bartholdi'
Finished processing : 1000 samples.
There are 541 samples that SUPPORTS.
There are 189 samples that REFUTES.
Time taken for last 500 iterations :  112.17393589019775
Finished processing : 1500 samples.
There are 784 samples that SUPPORTS.
There are 306 samples that REFUTES.
Time taken for last 500 iterations :  111.08995008468628
'Björn_Borg'
Finished processing : 2000 samples.
There are 1070 samples that SUPPORTS.
There are 399 samples that REFUTES.
Time taken for last 500 iterations :  120.7069571018219
'Citadelle_Laferrière'
Finished processing : 2500 samples.
There are 1334 samples that SUPPORTS.
There are 500 samples that REFUTES.
Time taken for last 500 iterations :  117.71129989624023


KeyboardInterrupt: 

In [23]:
not (num_supported==max_num_supported) and not  (num_refuted==max_num_refuted)

True